In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder

# --- Cargar y preparar los datos ---
# Cargar dataset Iris
iris = load_iris()
data = pd.DataFrame(iris.data, columns=iris.feature_names)
data['species'] = iris.target

# Filtrar solo las clases 'setosa' (0) y 'virginica' (2)
data = data[data['species'].isin([0, 2])]

# Separar características (X) y etiquetas (y)
X = data.iloc[:, :-1].values
y = data['species'].values

# Convertir las etiquetas a -1 (setosa) y 1 (virginica)
y = np.where(y == 0, -1, 1)

# --- Dividir los datos en Hold-Out 70/30 ---
np.random.seed(42)  # Asegurar reproducibilidad
indices = np.random.permutation(len(X))
split = int(len(X) * 0.7)

train_indices = indices[:split]
test_indices = indices[split:]

X_train, y_train = X[train_indices], y[train_indices]
X_test, y_test = X[test_indices], y[test_indices]

# --- Perceptrón ---
class Perceptron:
    def __init__(self, learning_rate=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        # Inicializar pesos y sesgo
        n_features = X.shape[1]
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Entrenamiento
        for _ in range(self.epochs):
            for i in range(len(X)):
                # Calcular la salida del perceptrón
                linear_output = np.dot(X[i], self.weights) + self.bias
                prediction = 1 if linear_output >= 0 else -1

                # Actualizar pesos y sesgo si hay error
                if y[i] != prediction:
                    self.weights += self.learning_rate * y[i] * X[i]
                    self.bias += self.learning_rate * y[i]

    def predict(self, X):
        # Realizar predicciones
        linear_output = np.dot(X, self.weights) + self.bias
        return np.where(linear_output >= 0, 1, -1)

# --- Entrenamiento y validación ---
# Crear y entrenar el perceptrón
perceptron = Perceptron(learning_rate=0.01, epochs=1000)
perceptron.fit(X_train, y_train)

# Evaluar en el conjunto de prueba
y_pred = perceptron.predict(X_test)

# Calcular precisión manualmente
accuracy = np.mean(y_pred == y_test)
print(f"Precisión en el conjunto de prueba (Hold-Out 70/30): {accuracy:.2f}")

# --- Resultados ---
# Mostrar pesos, sesgo, y matriz de confusión manual
print(f"Pesos aprendidos: {perceptron.weights}")
print(f"Sesgo aprendido: {perceptron.bias}")

# Crear matriz de confusión
confusion_matrix = np.zeros((2, 2), dtype=int)
for true, pred in zip(y_test, y_pred):
    i = 0 if true == -1 else 1
    j = 0 if pred == -1 else 1
    confusion_matrix[i, j] += 1

print("Matriz de confusión:")
print(confusion_matrix)


Precisión en el conjunto de prueba (Hold-Out 70/30): 1.00
Pesos aprendidos: [-0.02  -0.054  0.067  0.03 ]
Sesgo aprendido: -0.01
Matriz de confusión:
[[11  0]
 [ 0 19]]
